In [ ]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    model_complexity=1,
    min_detection_confidence=.9,
    min_tracking_confidence=.9,
)

def get_landmarks(image):
    results = hands.process(image)

    # If hand not detected, retry, this time without
    # tracking landmarks from previous image
    if not results.multi_hand_landmarks:
        results = hands.process(image)

    return results.multi_hand_landmarks

def landmark_image(image, landmarks):
    for hand_landmarks in landmarks:
        mp_drawing.draw_landmarks(
            image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
        )

def annotate_image(image):
    landmarks = get_landmarks(image)
    if landmarks:
        landmark_image(image, landmarks)
    return image

def annotate_black_image(image):
    landmarks = get_landmarks(image)
    res = np.zeros_like(image)
    if landmarks:
        landmark_image(res, landmarks)
    return res

def get_video_info(video_path):
    vc = cv2.VideoCapture(video_path)

    ret = {
        'width': int(vc.get(cv2.CAP_PROP_FRAME_WIDTH)),
        'height': int(vc.get(cv2.CAP_PROP_FRAME_HEIGHT)),
        'frames': int(vc.get(cv2.CAP_PROP_FRAME_COUNT)),
        'fps': vc.get(cv2.CAP_PROP_FPS),
        'fourcc': int(vc.get(cv2.CAP_PROP_FOURCC)),
    }

    vc.release()
    return ret

def read_video(video_path):
    vc = cv2.VideoCapture(video_path)
    is_portrait = vc.get(cv2.CAP_PROP_FRAME_WIDTH) < vc.get(cv2.CAP_PROP_FRAME_HEIGHT)
    while vc.grab():
        bgr = vc.retrieve()[1]
        if is_portrait:
            bgr = cv2.rotate(bgr, cv2.ROTATE_180)
        yield cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
    vc.release()

def write_video(video_path, video_info, video):
    vw = cv2.VideoWriter(
        video_path,
        video_info['fourcc'],
        video_info['fps'],
        (video_info['width'], video_info['height'])
    )
    for frame in video:
        bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        vw.write(bgr)
    vw.release()

def process_video(video_filename):
    video_path = f'video_in/{video_filename}'
    video_info = get_video_info(video_path)
    video = read_video(video_path)

    #write_video(f'video_out/{video_filename}', video_info, map(annotate_image, video))
    write_video(f'video_out/{video_filename}', video_info, map(annotate_black_image, video))

process_video('ID32_02-12-2021_DCHA (H).mp4')